In [1]:
#!/usr/bin/env python3
import tensorflow as tf

import os

import matplotlib.pyplot as plt 
import matplotlib
import matplotlib.gridspec as gridspec
import matplotlib.dates as mdates
from matplotlib.ticker import FixedLocator, FixedFormatter, FuncFormatter, MultipleLocator

import numpy as np
import pandas as pd

from src.ForecastModel.data.models import DataModelCV
from src.ForecastModel.utils.losses import loss_peak_mse
from src.ForecastModel.utils.metrics import (evaluate_multistep,
                                             calculate_rms, calculate_bias, 
                                             calculate_bias_flv, calculate_bias_fhv,
                                            )
from src.ForecastModel.utils.postprocessing import ModelHandler

plt.rcParams.update({
    "text.usetex": False,
    'font.size'   : 8,
})


In [2]:
def dt(dates, format="%d/%m/%Y %H:%M"):
    if dates.tz == None:
        # make TZ aware
        return pd.to_datetime(dates, format=format).tz_localize("Europe/London").tz_convert("UTC")
    else:
        return pd.to_datetime(dates, format=format).tz_convert("UTC")
   

In [3]:
PLOT_PATH          = r"plots"
DATA_PATH          = r"src\data\Dataset.csv"
CROSS_INDICES_PATH = r"src\data\indices"

In [4]:
models = {
    "arima": ModelHandler("ARIMA",
                r"src\rst\ARIMA",
                is_final_model = True,
                is_external_model = True,
                color = "#E69F00",
                ls = "--",
                  ),
    "arimax": ModelHandler("ARIMAX",
                r"src\rst\ARIMAX",
                is_final_model = True,
                is_external_model = True,
                color = "#0072B2",
                ls = "--",
                  ),
     "pbhm-hlstm": ModelHandler("PBHM-HLSTM",
                   r"src\rst\PBHM-HLSTM",
                   is_final_model = True,
                   color = "#56B4E9",
                   ls = "-",
                 ),
     "elstm": ModelHandler("eLSTM",
                   r"src\rst\eLSTM",
                   is_final_model = True,
                   color = "#D55E00",
                   ls = "-",
                 ),
     "lstm": ModelHandler("LSTM",
                   r"src\rst\LSTM",
                   is_final_model = True,
                   color = "#CC79A7",
                   ls = "-",
                 ),
     }


In [6]:
# define metrics to evaluate
eval_metrics = {
        "pbias" : calculate_bias,
        "fhv" : calculate_bias_fhv,
        "flv" : calculate_bias_flv,
    }

idx = -10
dfp = pd.DataFrame()
for n, key in enumerate(models.keys()):
    idx += 10
    print(key)

    metrics = {
        "valid": {}, 
        "test" : {},
    }
    for k, item in eval_metrics.items():
        metrics["test"][k] = []

    eval_path = os.path.join(models[key].hp_path, "eval_peaks.pckl")


    if models[key].is_external_model:
        overlap_length = 0
        hindcast_length = 96
    else:
        # load datamodel
        dm = DataModelCV(DATA_PATH,
           target_name       = models[key].target_name,
           hincast_features  = models[key].feat_hindcast,
           forecast_features = models[key].feat_forecast,
         )
        
        # load trial data
        with open(os.path.join(models[key].hp_path, "trial.json")) as f:
            trial = json.load(f)

        hindcast_length = trial['hyperparameters']['values']['hindcast_length']
        try:
            overlap_length = trial['hyperparameters']['values']['osc_length']
        except:
            overlap_length = 0 
        
        dm.main(os.path.join(CROSS_INDICES_PATH, f"cross_indices_{hindcast_length}.pckl"))

    for n_fold in range(5):
        year = 2013 + n_fold
        if models[key].is_external_model:
            # load external which come already with observations
            ext_df = pd.read_pickle(os.path.join(models[key].hp_path, f"forecast_{year}.pckl"))

            # get external model observations
            y = np.expand_dims(ext_df.filter(like="obs").values, axis=2) # fix 

            # get external model predictions
            yp = ext_df.filter(like="fc").values
                        
        else:
            # load dataset
            X, y  = dm.getDataSet(dm.cross_sets[n_fold]["test"], scale=True) 

            # load model
            tf.keras.backend.clear_session()
            model  = tf.keras.models.load_model(os.path.join(models[key].hp_path, f"model_fold_{n_fold:d}.keras"),
                                           custom_objects={'peak_loss'     : loss_peak_mse, # dummy as no costum functions are saved by keras
                                                           'kge_nse_loss'  : loss_peak_mse, #
                                                           'loss_nkge_nnse': loss_peak_mse, #
                                                          })
            # model prediction
            yp = model.predict(X, batch_size=1000)

        for k, item in eval_metrics.items():
            metrics["test"][k].append(evaluate_multistep(y, yp, item))

    with open(os.path.join(models[key].hp_path, f"metrics_eval.txt"), "w+") as f:
        json.dump(metrics, f)
            

arima
arimax
pbhm-hlstm
dictonary loaded
35/35 [==============================] - 1s 16ms/step
elstm
dictonary loaded
35/35 [==============================] - 1s 11ms/step
lstm
dictonary loaded
35/35 [==============================] - 0s 7ms/step
